# 基本设置

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2

In [2]:
from toolkits.setup import specific_func
specific_func.set_ch_pd()
from toolkits.setup.date_time import get_day_list
from toolkits.nlp import pre_cor_circ
from toolkits.nlp import pre_cor_cbrc
from toolkits.nlp import myclass_circ
from toolkits.nlp import myclass_cbrc

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.085 seconds.
Prefix dict has been built succesfully.


In [35]:
import requests,json

In [31]:
def get_server_res(data, url, col_name):
    '''
    服务器接口测试程序
    传入 dict, 传出 DataFrame
    '''
    # data = {'record':[{'id':0,'title':'ss','content':'zzz'},]}
    # data = {"record":marked_human_data.iloc[:5,:3].to_dict(orient = 'records')}
    # url "http://47.93.77.19:10000/correlation_negative"
    headers={'content-type':'application/json'}
    result = requests.post(url,
                      data = json.dumps(data),
                      headers=headers, allow_redirects=True)
    # print(result.text)
    json_data = json.loads(result.text)
    parse_data = []
    elapsed_time = json_data['elapsed_time']
    for i in range(len(json_data['docs'])):
        parse_data.append([json_data['docs'][i]['id'],
                          json_data['docs'][i][col_name]])
    parse_data = pd.DataFrame(parse_data, columns = ['id', col_name])    
    return parse_data, elapsed_time

In [46]:
def get_server_res_yjh(data, url, col_name):
    '''
    服务器接口测试程序
    传入 dict, 传出 DataFrame
    '''
    # data = {'record':[{'id':0,'title':'ss','content':'zzz'},]}
    # data = {"record":marked_human_data.iloc[:5,:3].to_dict(orient = 'records')}
    # url "http://47.93.77.19:10000/correlation_negative"
    headers={'content-type':'application/json'}
    result = requests.post(url,
                      data = json.dumps(data),
                      headers=headers, allow_redirects=True)
    # print(result.text)
    json_data = json.loads(result.text)
    parse_data = []
    elapsed_time = json_data['elapsed_time']
    for i in range(len(json_data['docs'])):
        parse_data.append([json_data['docs'][i]['id'],
                          json_data['docs'][i][col_name]])
    parse_data = pd.DataFrame(parse_data, columns = ['id', col_name])    
    return parse_data , elapsed_time

# load data

In [72]:
data_test = pd.read_excel('20190108_验证补录数据/银保监、国寿、建行采到没有入库的数据.xlsx')
print('去空值前：', data_test.shape)
data_test = data_test.dropna()
print('去空值后：', data_test.shape)
data_test['url'] = data_test['链接'].apply(lambda x:x.replace('//', '://'))
data_test.iloc[:1, :]

去空值前： (1877, 6)
去空值后： (1875, 6)


,index,标题,链接,时间,时间2,项目,url
0,1,超30家壽險公司三季度虧損 四季度保費有望扭轉頹勢,http//big5.china.com.cn/gate/big5/finance.china.com.cn/money/insurance/20181106/4801668.shtml,20181106091800,2018,保监会舆情CIRC,http://big5.china.com.cn/gate/big5/finance.china.com.cn/money/insurance/20181106/4801668.shtml


In [73]:
data_test['项目'].value_counts()

银监会(15号)舆情    1651
保监会舆情CIRC      121
建行北京分行舆情        74
中国人寿舆情项目        29
Name: 项目, dtype: int64

In [74]:
data_cbrc = data_test[data_test['项目'].str.contains('银监会')]
print('data_cbrc: ', data_cbrc.shape)
data_circ = data_test[data_test['项目'].str.contains('保监会')]
print('data_circ: ', data_circ.shape)
data_ccb = data_test[data_test['项目'].str.contains('建行北京分行')]
print('data_ccb: ', data_ccb.shape)
data_clic = data_test[data_test['项目'].str.contains('中国人寿')]
print('data_clic: ', data_clic.shape)

data_cbrc:  (1651, 7)
data_circ:  (121, 7)
data_ccb:  (74, 7)
data_clic:  (29, 7)


# 调用接口

In [6]:
label_dic={'补录':0,'监管':1,'行业':2,'产品销售':3,'资本市场':4,'公司内部管理':5,
           '消费服务':6,'其他相关报道':7,'噪音':8,'交通':9,'环保':10}
class_name_dict = {v: k for k, v in label_dic.items()}
class_name_dict

{0: '补录',
 1: '监管',
 2: '行业',
 3: '产品销售',
 4: '资本市场',
 5: '公司内部管理',
 6: '消费服务',
 7: '其他相关报道',
 8: '噪音',
 9: '交通',
 10: '环保'}

In [7]:
gather_type_dic={'系统采集':0,'补录':1,'校正':2,'导入数据':3,'其它':4}
gather_type_name_dict = {v: k for k, v in gather_type_dic.items()}
gather_type_name_dict

{0: '系统采集', 1: '补录', 2: '校正', 3: '导入数据', 4: '其它'}

## 保监会

In [29]:
engine = specific_func.get_engine('circ')

In [41]:
url_tuple = tuple(data_circ['url'].tolist())

sql_one_day = "select t1.id, t1.gather_type,t1.url,t1.title,t2.center as content, t1.publishtime as publishtime \
                    from wise_web_docinfo t1, wise_web_docinfo_center t2 \
                        where t1.id=t2.doc_id \
                              and  t1.url in {0}".format(url_tuple) # 

circ_cor = pd.read_sql(sql_one_day, engine)
circ_cor['gather_type'] = circ_cor['gather_type'].apply(lambda x:gather_type_name_dict[x])
print('circ_cor: ', circ_cor.shape)
circ_cor.iloc[:1, :]

circ_cor:  (90, 6)


,id,gather_type,url,title,content,publishtime
0,12581783,补录,https://m.thepaper.cn/newsDetail_forward_2705527,中国人寿一营销员发布自媒体文章虚假宣传，被罚6000元,由于发布自媒体文章进行虚假宣传及承诺，中国人寿海南省分公司一保险营销员被罚6000元。\n12月4日，海南银保监局筹备组发布的一则行政处罚决定书显示，2018年，中国人寿保险股份有限公司海南省分公司保险营销员王小伟通过自己注册的微信公众号“海口车险”发布文章《买车险，高优惠（海口车主都传疯了......）》进行虚假宣传及承诺。与此同时，王小伟还主动联系某电台广告推广人员，将该文章在某电台微信公众号进行推广。\n海南银保监局筹备组表示，王小伟的行为违反了《中华人民共和国保险法》中有关保险代理人、保险经纪人及其从业人员在办理保险业务活动中的禁止行为。上述违法行为，有现场检查事实确认书、调查笔录、微信公众号“海口车险”相关内容截图材料等证据证明，足以认定。\n对此，王小伟提出陈述申辩，申请从轻处罚：一是推广公众号2小时后，其已将不实宣传文章删除并停止推广，同时积极配合检查组的调查及按要求注销“海口车险”公众号；二是其从事保险行业时间短、收入低、经济困难。\n但海南银保监局筹备组并未采纳他的申辩意见，决定对其警告并罚款6000元。海南银保监局筹备组表示，王小伟陈述提到的删除文章、停止推广、积极配合检查等事实已在拟作出行政处罚时予以考虑；王小伟的陈述申辩中，对处罚事实和证据没有异议，且没有提出新的证据。\n针对自媒体营销可能产生的一系列违规问题，银保监会早在今年6月便发文要求各保险公司、保险中介机构对本公司官方自媒体和所属保险从业人员个人自媒体情况展开自查。究其原因，银保监会表示，由于自媒体渠道参与门槛低、发布主体多、信息审核弱、转发传播快，已成为保险销售误导、不实信息传播的高发领域，严重损害保险消费者合法权益，埋下大量保险消费纠纷和群体性事件风险隐患。\n6月4日，银保监会下发《关于加强自媒体保险营销宣传行为管理的通知》（下称《通知》），要求严格规范所属保险从业人员利用自媒体编发保险营销宣传信息行为，对于涉及保险产品介绍、销售政策和营销宣传推介活动的，应以公司官方自媒体信息为准，严禁保险公司分支机构、保险中介机构分级机构及保险从业人员自行编发，对于不涉及上述内容的，应当制定责任部门进行审核管理。在进行保险营销宣传信息转载时，严禁转载未经所在公司审核的内容，确保真实可靠以及信息源可追溯。\n在责任追究方面，各保险公司、保险中介机构应当将违法和不当自媒体保险营销宣传行为...,2018-12-05 08:35:27


In [42]:
data = {"record":circ_cor.loc[:,['id', 'title' ,'content']].to_dict(orient = 'records')}
url = "http://47.93.77.19:10000/judge_correlation_i"
col_name = 'cor'
parse_data, elapsed_time = get_server_res(data, url, col_name)
parse_data.columns = ['id', 'predict_label']
print(parse_data.shape)
parse_data.head()

(90, 2)


,id,predict_label
0,12581783,5
1,12716633,4
2,12716640,4
3,12716662,8
4,12716664,8


In [45]:
circ_combined = pd.merge(circ_cor, parse_data, on = 'id')
circ_combined['predict_label'] = circ_combined['predict_label'].apply(lambda x:class_name_dict[x])
print(circ_combined.shape)
print(circ_combined['predict_label'].value_counts())
circ_combined.iloc[:1, :]

(90, 7)
公司内部管理    31
噪音        31
行业        13
资本市场      10
消费服务       3
产品销售       1
监管         1
Name: predict_label, dtype: int64


,id,gather_type,url,title,content,publishtime,predict_label
0,12581783,补录,https://m.thepaper.cn/newsDetail_forward_2705527,中国人寿一营销员发布自媒体文章虚假宣传，被罚6000元,由于发布自媒体文章进行虚假宣传及承诺，中国人寿海南省分公司一保险营销员被罚6000元。\n12月4日，海南银保监局筹备组发布的一则行政处罚决定书显示，2018年，中国人寿保险股份有限公司海南省分公司保险营销员王小伟通过自己注册的微信公众号“海口车险”发布文章《买车险，高优惠（海口车主都传疯了......）》进行虚假宣传及承诺。与此同时，王小伟还主动联系某电台广告推广人员，将该文章在某电台微信公众号进行推广。\n海南银保监局筹备组表示，王小伟的行为违反了《中华人民共和国保险法》中有关保险代理人、保险经纪人及其从业人员在办理保险业务活动中的禁止行为。上述违法行为，有现场检查事实确认书、调查笔录、微信公众号“海口车险”相关内容截图材料等证据证明，足以认定。\n对此，王小伟提出陈述申辩，申请从轻处罚：一是推广公众号2小时后，其已将不实宣传文章删除并停止推广，同时积极配合检查组的调查及按要求注销“海口车险”公众号；二是其从事保险行业时间短、收入低、经济困难。\n但海南银保监局筹备组并未采纳他的申辩意见，决定对其警告并罚款6000元。海南银保监局筹备组表示，王小伟陈述提到的删除文章、停止推广、积极配合检查等事实已在拟作出行政处罚时予以考虑；王小伟的陈述申辩中，对处罚事实和证据没有异议，且没有提出新的证据。\n针对自媒体营销可能产生的一系列违规问题，银保监会早在今年6月便发文要求各保险公司、保险中介机构对本公司官方自媒体和所属保险从业人员个人自媒体情况展开自查。究其原因，银保监会表示，由于自媒体渠道参与门槛低、发布主体多、信息审核弱、转发传播快，已成为保险销售误导、不实信息传播的高发领域，严重损害保险消费者合法权益，埋下大量保险消费纠纷和群体性事件风险隐患。\n6月4日，银保监会下发《关于加强自媒体保险营销宣传行为管理的通知》（下称《通知》），要求严格规范所属保险从业人员利用自媒体编发保险营销宣传信息行为，对于涉及保险产品介绍、销售政策和营销宣传推介活动的，应以公司官方自媒体信息为准，严禁保险公司分支机构、保险中介机构分级机构及保险从业人员自行编发，对于不涉及上述内容的，应当制定责任部门进行审核管理。在进行保险营销宣传信息转载时，严禁转载未经所在公司审核的内容，确保真实可靠以及信息源可追溯。\n在责任追究方面，各保险公司、保险中介机构应当将违法和不当自媒体保险营销宣传行为...,2018-12-05 08:35:27,公司内部管理


## 银监会

In [47]:
engine = specific_func.get_engine('cbrc')

In [48]:
url_tuple = tuple(data_cbrc['url'].tolist())

sql_one_day = "select t1.id, t1.gather_type, t1.url, t1.title,t2.text as content, t1.publishtime as publishtime \
                    from elint_web_docinfo t1, wise_web_docinfo_text t2 \
                        where t1.id=t2.doc_id \
                            and  t1.url in {0}".format(url_tuple)

cbrc_cor = pd.read_sql(sql_one_day, engine)
cbrc_cor['gather_type'] = cbrc_cor['gather_type'].apply(lambda x:gather_type_name_dict[x])
print('cbrc_cor: ', cbrc_cor.shape)
cbrc_cor.iloc[:1, :]

cbrc_cor:  (720, 6)


,id,gather_type,url,title,content,publishtime
0,207398163,系统采集,http://finance1.ce.cn/rolling/201801/01/t20180101_27507758.shtml,2018保险业：三大难题悬而未决,雄关漫道真如铁，而今迈步从头越。2018年，中国保险业前路艰辛，三道难题摆在面前：“开门红”难有作为，全年保费目标如何完成？“大监管”有增无减，合规成本如何控制？BATJ强势“搅局”，金融科技有何作为？“开门红”集体哑火“开门红，全年红”，这是保险业的一条“金科玉律”。2018年，这条“铁律”可能要被打破了。年底保险圈内聚会上，大佬们见面寒暄的第一句话是：“你们今年‘开门红’卖得怎么样？”一部分人面露难色，一部分频频摇头，还有些干脆用一句“不谈了”草草了事。2018年，“开门红”为什么集体哑火？一家大型寿险公司负责人在接受《国际金融报》记者采访时透露，“一方面，受134号文（《关于规范人身保险公司产品开发设计行为的通知》）和136号文（《关于进一步加强人身保险公司销售管理工作的通知》）影响，今年各家公司‘开门红’启动时间比较晚。另一方面，今年‘开门红’产品回归保障，规模上不去在情理之中。”这两个文件究竟对“开门红”有什么影响？对此，一位资深保险业内人士告诉《国际金融报》记者：“134号文主要针对产品设计，136号文则针对产品销售。相比之下，前者的影响更大。在134号文提出的2017年10月1日‘大限’之日，很多保险公司都被迫更新产品，这些产品都是计划作为‘开门红’的产品。但是，10月1日真正拿到批文的产品没几个。”134号文核心内容包括：两全保险产品、年金保险产品，首次生存保险金给付应在保单生效满5年之后，且每年给付或部分领取比例不得超过已交保险费的20%；万能型保险产品、投资连结型保险产品设计应提供不定期、不定额追加保险费，灵活调整保险金额等功能。保险公司不得以附加险形式设计万能型保险产品或投资连结型保险产品。“我们的‘开门红’产品刚刚批下来。”2017年12月28日，一家合资寿险公司有关负责人向《国际金融报》记者透露，“往年10月至11月是启动‘开门红’的重要节点，今年，整个行业的‘开门红’时间都有所推迟。这跟保监会产品审批趋严有关。”一家大型寿险公司上海分公司总经理告诉《国际金融报》记者：“我们直到去年12月才收到总公司对‘开门红’的安排以及目标任务。坦率地说，市场都比较谨慎，大家都看不清楚今年‘开门红’的趋势，也没有把握。”那么，最早一批启动“开门红”的公司，业绩表现如何？一家上市保险公司上海分公司有关负责人在接受《国际金融报》记者采访时说：“今年‘...,2018-01-01 08:22:34


In [55]:
data = {"record":cbrc_cor.loc[:,['id', 'title' ,'content']].to_dict(orient = 'records')}
url = "http://47.93.77.19:6001/judge_correlation_yjh"
col_name = 'sec'
parse_data, elapsed_time = get_server_res(data, url, col_name)
parse_data.columns = ['id', 'predict_label']
print(parse_data.shape)
parse_data.head()

(720, 2)


,id,predict_label
0,207398163,1
1,207598948,1
2,207633488,2
3,207637910,1
4,207640544,1


In [56]:
cbrc_combined = pd.merge(cbrc_cor, parse_data, on = 'id')
cbrc_combined['predict_label'] = cbrc_combined['predict_label'].apply(lambda x:class_name_dict[x])
print(cbrc_combined.shape)
print(cbrc_combined['predict_label'].value_counts())
cbrc_combined.iloc[:1, :]

(720, 7)
行业        186
消费服务      181
监管         97
公司内部管理     88
噪音         84
资本市场       78
其他相关报道      4
产品销售        2
Name: predict_label, dtype: int64


,id,gather_type,url,title,content,publishtime,predict_label
0,207398163,系统采集,http://finance1.ce.cn/rolling/201801/01/t20180101_27507758.shtml,2018保险业：三大难题悬而未决,雄关漫道真如铁，而今迈步从头越。2018年，中国保险业前路艰辛，三道难题摆在面前：“开门红”难有作为，全年保费目标如何完成？“大监管”有增无减，合规成本如何控制？BATJ强势“搅局”，金融科技有何作为？“开门红”集体哑火“开门红，全年红”，这是保险业的一条“金科玉律”。2018年，这条“铁律”可能要被打破了。年底保险圈内聚会上，大佬们见面寒暄的第一句话是：“你们今年‘开门红’卖得怎么样？”一部分人面露难色，一部分频频摇头，还有些干脆用一句“不谈了”草草了事。2018年，“开门红”为什么集体哑火？一家大型寿险公司负责人在接受《国际金融报》记者采访时透露，“一方面，受134号文（《关于规范人身保险公司产品开发设计行为的通知》）和136号文（《关于进一步加强人身保险公司销售管理工作的通知》）影响，今年各家公司‘开门红’启动时间比较晚。另一方面，今年‘开门红’产品回归保障，规模上不去在情理之中。”这两个文件究竟对“开门红”有什么影响？对此，一位资深保险业内人士告诉《国际金融报》记者：“134号文主要针对产品设计，136号文则针对产品销售。相比之下，前者的影响更大。在134号文提出的2017年10月1日‘大限’之日，很多保险公司都被迫更新产品，这些产品都是计划作为‘开门红’的产品。但是，10月1日真正拿到批文的产品没几个。”134号文核心内容包括：两全保险产品、年金保险产品，首次生存保险金给付应在保单生效满5年之后，且每年给付或部分领取比例不得超过已交保险费的20%；万能型保险产品、投资连结型保险产品设计应提供不定期、不定额追加保险费，灵活调整保险金额等功能。保险公司不得以附加险形式设计万能型保险产品或投资连结型保险产品。“我们的‘开门红’产品刚刚批下来。”2017年12月28日，一家合资寿险公司有关负责人向《国际金融报》记者透露，“往年10月至11月是启动‘开门红’的重要节点，今年，整个行业的‘开门红’时间都有所推迟。这跟保监会产品审批趋严有关。”一家大型寿险公司上海分公司总经理告诉《国际金融报》记者：“我们直到去年12月才收到总公司对‘开门红’的安排以及目标任务。坦率地说，市场都比较谨慎，大家都看不清楚今年‘开门红’的趋势，也没有把握。”那么，最早一批启动“开门红”的公司，业绩表现如何？一家上市保险公司上海分公司有关负责人在接受《国际金融报》记者采访时说：“今年‘...,2018-01-01 08:22:34,监管


## 中国人寿 3

In [57]:
engine = specific_func.get_engine('cbirc')

In [58]:
url_tuple = tuple(data_clic['url'].tolist())
url_tuple = tuple([url.replace('%', '%%') for url in url_tuple])

sql_one_day = '''SELECT 
    t1.id, t2.gather_type, t1.url, t1.title, t3.text as content, t1.publishtime
FROM
    cbrc_circ.db_docinfo t1,
    cbrc_circ.db_docinfo_trade t2, 
    cbrc_circ.db_docinfo_text t3
WHERE
    t2.type = 3
        and t1.url IN {0}
        and t1.urlhash = t2.urlhash
        and t1.urlhash = t3.urlhash
        '''.format(url_tuple)

clic_cor = pd.read_sql(sql_one_day, engine)
clic_cor['gather_type'] = clic_cor['gather_type'].apply(lambda x:gather_type_name_dict[x])
print('clic_cor: ', clic_cor.shape)
clic_cor.iloc[:1, :]

clic_cor:  (21, 6)


,id,gather_type,url,title,content,publishtime
0,103261546,补录,http://bbs.suqian.cn/forum.php?mod=viewthread&tid=660316&extra=page%3D1,【咨询投诉】 关于宿城区中国人寿保险分公司未理赔的问题 [复制链接],关于宿城区中国人寿保险分公司未理赔的问题 - 问政论坛 - 宿迁12345 - 便民服务365 - Powered by Discuz!\r\n开启辅助访问\r\n请 登录 后使用快捷导航没有帐号？立即注册\r\n﻿\nAPP下载\n|\n登录\n|\n注册\n ﻿\r\n搜索\r\n本版文章帖子用户\r\n首页网站首页\r\n问政论坛\r\n政民互动\r\n宿迁12345 - 便民服务365?首页 &rsaquo; 宿迁12345 &rsaquo; 问政论坛 &rsaquo; 关于宿城区中国人寿保险分公司未理赔的问题 ...\r\n查看详情\n发帖\r\n受理\r\n办结\r\n发帖\r\n受理\r\n办结\r\n1、处理人：\r\n188608*****\r\n处理时间：\r\n2018-10-27 03:46:35\r\n发帖,2018-10-27 03:46:35


In [60]:
url = 'http://47.93.183.157:10000/judge_correlation_i'
col_name = 'cor'
types = 3
data = {"types":types,"record":clic_cor.loc[:,['id', 'title' ,'content']].to_dict(orient = 'records')}
parse_data, elapsed_time = get_server_res(data, url, col_name)
print('elapsed_time: ', elapsed_time)
parse_data.columns = ['id', 'predict_label']
print(parse_data.shape)
parse_data.head()

elapsed_time:  0.00
(21, 2)


,id,predict_label
0,103261546,6
1,81827252,5
2,81821923,8
3,82562137,2
4,87306251,2


In [61]:
clic_combined = pd.merge(clic_cor, parse_data, on = 'id')
clic_combined['predict_label'] = clic_combined['predict_label'].apply(lambda x:class_name_dict[x])
print(clic_combined.shape)
print(clic_combined['predict_label'].value_counts())
clic_combined.iloc[:1, :]

(21, 7)
噪音        11
消费服务       3
公司内部管理     3
行业         2
监管         1
资本市场       1
Name: predict_label, dtype: int64


,id,gather_type,url,title,content,publishtime,predict_label
0,103261546,补录,http://bbs.suqian.cn/forum.php?mod=viewthread&tid=660316&extra=page%3D1,【咨询投诉】 关于宿城区中国人寿保险分公司未理赔的问题 [复制链接],关于宿城区中国人寿保险分公司未理赔的问题 - 问政论坛 - 宿迁12345 - 便民服务365 - Powered by Discuz!\r\n开启辅助访问\r\n请 登录 后使用快捷导航没有帐号？立即注册\r\n﻿\nAPP下载\n|\n登录\n|\n注册\n ﻿\r\n搜索\r\n本版文章帖子用户\r\n首页网站首页\r\n问政论坛\r\n政民互动\r\n宿迁12345 - 便民服务365?首页 &rsaquo; 宿迁12345 &rsaquo; 问政论坛 &rsaquo; 关于宿城区中国人寿保险分公司未理赔的问题 ...\r\n查看详情\n发帖\r\n受理\r\n办结\r\n发帖\r\n受理\r\n办结\r\n1、处理人：\r\n188608*****\r\n处理时间：\r\n2018-10-27 03:46:35\r\n发帖,2018-10-27 03:46:35,消费服务


## 建行北分

In [62]:
url_tuple = tuple(data_ccb['url'].tolist())
url_tuple = tuple([url.replace('%', '%%') for url in url_tuple])

sql_one_day = '''SELECT 
    t1.id, t2.gather_type, t1.url, t1.title, t3.text as content, t1.publishtime
FROM
    cbrc_circ.db_docinfo t1,
    cbrc_circ.db_docinfo_trade t2, 
    cbrc_circ.db_docinfo_text t3
WHERE
    t2.type = 4
        and t1.url IN {0}
        and t1.urlhash = t2.urlhash
        and t1.urlhash = t3.urlhash
        '''.format(url_tuple)

ccb_cor = pd.read_sql(sql_one_day, engine)
ccb_cor['gather_type'] = ccb_cor['gather_type'].apply(lambda x:gather_type_name_dict[x])
print('ccb_cor: ', ccb_cor.shape)
ccb_cor.iloc[:1, :]

ccb_cor:  (52, 6)


,id,gather_type,url,title,content,publishtime
0,84505510,补录,http://house.qq.com/a/20180918/002706.htm,长租市场迎来国家队：银行进军租房市场能解决痛点吗?,近期租房市场可谓处在风口浪尖……细心观察，租房市场一系列尖锐矛盾的背后都离不开资本这把双刃剑。那么，当最擅长资本游戏的银行亲自披挂上阵，深度参与房屋租赁交易时，租客和业主们又会迎来怎样一番景象呢？\n\r要租房找银行\n\r白领丽人小叶在北京参加工作已有四年，目前在一家央企工作。入职以来，她屡屡听同事说单位要搬到河北，所以迟迟没有下决心在北京买房。最近，一直租房居住的她又因为房东要卖房不得不搬家，这几天为了找新房跟各路中介打交道，身心疲惫不堪。小叶远在武汉的同学小文建议她可以试试建设银行推出的“建融家园”APP，上面有不少房源，还可以方便获取贷款。小文就是通过“建融家园”APP在武汉租到了理想的房子。小叶听后颇为心动。\n\r她进入手机下载中心，发现有“CCB建融家园”和“CCB建融公寓”两款软件，前者有个人房源也有品牌公寓，后者主要是各种精品公寓。粗略看来，公寓房源远远多于个人房源。个人房源基本上都是与中介合作的，发布信息上都标注“政府验真”，北京地区三个月内只有36套。相比之下，品牌公寓的房源数量多、质量高，三个月内有1446套，大多来自金融街公寓、创业公社37度公寓、华贸公寓、丽都壹号、国贸世纪公寓、通泰文园、利锦新苑等。与其他房产中介APP一样，建融家园支持在线预约看房，不同的是租客可以直接在建融家园上签约租房，还可以利用该软件记录水电表度数、在线报修、下载智能门锁。\n\r小叶想在西城区找个1室1厅的房子。输入搜索条件后，她发现最贵的是长安兴融中心，100平方米的北向1室1厅，月租金5万元起。价格比较适中的有荣丰小区、一铭公寓等，1室1厅面积在40平方米左右，月租金3000元至4000元。小叶很快就选中了一套价格、户型、位置都很理想的房子，预约这周末去看房。小叶不禁感慨：“没想到银行还开发了租房软件，而且有这么多好房源，比房产中介给我推荐的性价比都高。”\n\r能存款也能存房\n\r相比北京地区，建融家园在南方城市的房源更加丰富。比如其官网9月6日公布的租房数据显示，上海地区最近三个月发布的房源是24988套，成都9574套，南京11578套，武汉9087套。同北京地区一样，建行在其他城市的房源也是品牌公寓的数量远远多于个人房源。\n\r为进一步拓展个人房源供给，今年1月，建行在广东地区首创推出了“家庭不动产财富管理”业务，亦称“存房”业务。所谓“...,2018-09-18 08:27:06


In [64]:
url = 'http://47.93.183.157:6001/judge_correlation_b'
col_name = 'cor'
types = 4
data = {"types":types,"record":ccb_cor.loc[:,['id', 'title' ,'content']].to_dict(orient = 'records')}
parse_data, elapsed_time = get_server_res(data, url, col_name)
print('elapsed_time: ', elapsed_time)
parse_data.columns = ['id', 'predict_label']
print(parse_data.shape)
parse_data.head()

elapsed_time:  1.00
(52, 2)


,id,predict_label
0,84505510,8
1,84908689,1
2,85043496,5
3,85043616,5
4,85056913,5


In [65]:
ccb_combined = pd.merge(ccb_cor, parse_data, on = 'id')
ccb_combined['predict_label'] = ccb_combined['predict_label'].apply(lambda x:class_name_dict[x])
print(ccb_combined.shape)
print(ccb_combined['predict_label'].value_counts())
ccb_combined.iloc[:1, :]

(52, 7)
噪音        23
消费服务      13
公司内部管理     8
其他相关报道     3
行业         2
监管         2
产品销售       1
Name: predict_label, dtype: int64


,id,gather_type,url,title,content,publishtime,predict_label
0,84505510,补录,http://house.qq.com/a/20180918/002706.htm,长租市场迎来国家队：银行进军租房市场能解决痛点吗?,近期租房市场可谓处在风口浪尖……细心观察，租房市场一系列尖锐矛盾的背后都离不开资本这把双刃剑。那么，当最擅长资本游戏的银行亲自披挂上阵，深度参与房屋租赁交易时，租客和业主们又会迎来怎样一番景象呢？\n\r要租房找银行\n\r白领丽人小叶在北京参加工作已有四年，目前在一家央企工作。入职以来，她屡屡听同事说单位要搬到河北，所以迟迟没有下决心在北京买房。最近，一直租房居住的她又因为房东要卖房不得不搬家，这几天为了找新房跟各路中介打交道，身心疲惫不堪。小叶远在武汉的同学小文建议她可以试试建设银行推出的“建融家园”APP，上面有不少房源，还可以方便获取贷款。小文就是通过“建融家园”APP在武汉租到了理想的房子。小叶听后颇为心动。\n\r她进入手机下载中心，发现有“CCB建融家园”和“CCB建融公寓”两款软件，前者有个人房源也有品牌公寓，后者主要是各种精品公寓。粗略看来，公寓房源远远多于个人房源。个人房源基本上都是与中介合作的，发布信息上都标注“政府验真”，北京地区三个月内只有36套。相比之下，品牌公寓的房源数量多、质量高，三个月内有1446套，大多来自金融街公寓、创业公社37度公寓、华贸公寓、丽都壹号、国贸世纪公寓、通泰文园、利锦新苑等。与其他房产中介APP一样，建融家园支持在线预约看房，不同的是租客可以直接在建融家园上签约租房，还可以利用该软件记录水电表度数、在线报修、下载智能门锁。\n\r小叶想在西城区找个1室1厅的房子。输入搜索条件后，她发现最贵的是长安兴融中心，100平方米的北向1室1厅，月租金5万元起。价格比较适中的有荣丰小区、一铭公寓等，1室1厅面积在40平方米左右，月租金3000元至4000元。小叶很快就选中了一套价格、户型、位置都很理想的房子，预约这周末去看房。小叶不禁感慨：“没想到银行还开发了租房软件，而且有这么多好房源，比房产中介给我推荐的性价比都高。”\n\r能存款也能存房\n\r相比北京地区，建融家园在南方城市的房源更加丰富。比如其官网9月6日公布的租房数据显示，上海地区最近三个月发布的房源是24988套，成都9574套，南京11578套，武汉9087套。同北京地区一样，建行在其他城市的房源也是品牌公寓的数量远远多于个人房源。\n\r为进一步拓展个人房源供给，今年1月，建行在广东地区首创推出了“家庭不动产财富管理”业务，亦称“存房”业务。所谓“...,2018-09-18 08:27:06,噪音


# 合并数据

In [66]:
combined_data = pd.concat([circ_combined, cbrc_combined, clic_combined, ccb_combined], axis = 0)
print(combined_data.shape)
print(combined_data['predict_label'].value_counts())
combined_data.iloc[:1, :]

(883, 7)
行业        203
消费服务      200
噪音        149
公司内部管理    130
监管        101
资本市场       89
其他相关报道      7
产品销售        4
Name: predict_label, dtype: int64


,id,gather_type,url,title,content,publishtime,predict_label
0,12581783,补录,https://m.thepaper.cn/newsDetail_forward_2705527,中国人寿一营销员发布自媒体文章虚假宣传，被罚6000元,由于发布自媒体文章进行虚假宣传及承诺，中国人寿海南省分公司一保险营销员被罚6000元。\n12月4日，海南银保监局筹备组发布的一则行政处罚决定书显示，2018年，中国人寿保险股份有限公司海南省分公司保险营销员王小伟通过自己注册的微信公众号“海口车险”发布文章《买车险，高优惠（海口车主都传疯了......）》进行虚假宣传及承诺。与此同时，王小伟还主动联系某电台广告推广人员，将该文章在某电台微信公众号进行推广。\n海南银保监局筹备组表示，王小伟的行为违反了《中华人民共和国保险法》中有关保险代理人、保险经纪人及其从业人员在办理保险业务活动中的禁止行为。上述违法行为，有现场检查事实确认书、调查笔录、微信公众号“海口车险”相关内容截图材料等证据证明，足以认定。\n对此，王小伟提出陈述申辩，申请从轻处罚：一是推广公众号2小时后，其已将不实宣传文章删除并停止推广，同时积极配合检查组的调查及按要求注销“海口车险”公众号；二是其从事保险行业时间短、收入低、经济困难。\n但海南银保监局筹备组并未采纳他的申辩意见，决定对其警告并罚款6000元。海南银保监局筹备组表示，王小伟陈述提到的删除文章、停止推广、积极配合检查等事实已在拟作出行政处罚时予以考虑；王小伟的陈述申辩中，对处罚事实和证据没有异议，且没有提出新的证据。\n针对自媒体营销可能产生的一系列违规问题，银保监会早在今年6月便发文要求各保险公司、保险中介机构对本公司官方自媒体和所属保险从业人员个人自媒体情况展开自查。究其原因，银保监会表示，由于自媒体渠道参与门槛低、发布主体多、信息审核弱、转发传播快，已成为保险销售误导、不实信息传播的高发领域，严重损害保险消费者合法权益，埋下大量保险消费纠纷和群体性事件风险隐患。\n6月4日，银保监会下发《关于加强自媒体保险营销宣传行为管理的通知》（下称《通知》），要求严格规范所属保险从业人员利用自媒体编发保险营销宣传信息行为，对于涉及保险产品介绍、销售政策和营销宣传推介活动的，应以公司官方自媒体信息为准，严禁保险公司分支机构、保险中介机构分级机构及保险从业人员自行编发，对于不涉及上述内容的，应当制定责任部门进行审核管理。在进行保险营销宣传信息转载时，严禁转载未经所在公司审核的内容，确保真实可靠以及信息源可追溯。\n在责任追究方面，各保险公司、保险中介机构应当将违法和不当自媒体保险营销宣传行为...,2018-12-05 08:35:27,公司内部管理


In [77]:
data_test_1 = pd.merge(data_test, combined_data, on = 'url', how = 'left')
data_test_1['gather_type'] = data_test_1['gather_type'].fillna('数据库中未查到')
print(data_test_1.shape)
print(data_test_1['predict_label'].value_counts())
print(data_test_1['gather_type'].value_counts())
data_test_1.iloc[:1, :]

(1879, 13)
行业        203
消费服务      200
噪音        153
公司内部管理    130
监管        102
资本市场       89
其他相关报道      7
产品销售        4
Name: predict_label, dtype: int64
数据库中未查到    991
补录         789
系统采集        94
校正           5
Name: gather_type, dtype: int64


,index,标题,链接,时间,时间2,项目,url,id,gather_type,title,content,publishtime,predict_label
0,1,超30家壽險公司三季度虧損 四季度保費有望扭轉頹勢,http//big5.china.com.cn/gate/big5/finance.china.com.cn/money/insurance/20181106/4801668.shtml,20181106091800,2018,保监会舆情CIRC,http://big5.china.com.cn/gate/big5/finance.china.com.cn/money/insurance/20181106/4801668.shtml,12759108.000,补录,超30家壽險公司三季度虧損 四季度保費有望扭轉頹勢,近期，壽險公司陸續公佈了今年三季度的償付能力報告，備受市場關注的是壽險公司虧損情況也隨之出爐。據《證券日報》記者不完全統計，今年三季度，共有超過30家險企凈利虧損(由於各險企披露口徑不統一，無法統計總體虧損金額)。\n\n 從虧損原因來看，在投資端，今年三季度，包括國壽股份等大型險企在內的不少險企，利潤受到權益類市場下跌的影響；從保費端來看，部分中小險企保障型保險暫未形成規模，死差益收入不足。\n\n 具體來看，三季度單季保險公司利潤增幅收窄，源於投資收益率同比下滑。前三季度中國平安、國壽股份、中國太保、新華保險年化總投資收益率分別為4.9%、3.3%、4.7%、4.8%，分別同比下降0.5%、1.8%、0.5%、0.4%。\n\n 雖然權益類投資收益不盡人意，拖累險企三季度凈利潤，但今年四季度保費端有望扭轉這一頹勢。\n\n 天風證券分析師羅鑽輝認為，下半年以來，銀行理財産品預期年化收益率大幅下滑，有利於保險産品的銷售。另外，儲蓄型産品和保障型産品的銷售存在著“共振效應”，儲蓄型産品的銷售改善也會提升保障型業務保費。\n\n 此外，從保險公司內部來看，2017年四季度保費基數較低，在前三季度保費仍有缺口的情況下，預計各家公司將大力推動四季度業務，全力達成年度目標，四季度新單保費和NBV將實現較快增長。長期來看，人身險保障缺口巨大，壽險深度仍低於全球平均水準的70%，壽險深度提升的過程中，隨著居民保險意識提升、每人平均可支配收入穩步提升、居民杠桿率的高位將在未來5年內提升人身險需求。\n\n 今年三季度，除部分險企出現虧損之外，其他險企還由於此前萬能險佔比較高，進入轉型期後，萬能險收入驟降，但保障型保險收入不足，死差益收入不足，持續出現虧損。\n\n 實際上，保險公司的利潤來源有三個：一是死差益，即保險條款預期的賠付額與實際的賠付額之間的差額；二是利差益，即保險條款預定的利率與保險公司實際投資收益之間的差額；三是費差益，即保險條款預定的費用和保險公司實際發生的費用之間的差額。\n\n 投射到保險産品，可將保險産品分為：保障性保險、投資性保險和保障投資性保險。保障性保險是指純保障性産品，沒有任何投資成分，短期意外和健康險偏向此類；投資性保險是指純投資性産品，沒有任何保障成分，投連險和萬能險偏向於此類；保障投資性保險是指既有保障成分又有投資...,2018-11-06 09:18:00,行业


In [79]:
writer = pd.ExcelWriter('银保监、国寿、建行_数据分析_20190110.xlsx',
                        engine='xlsxwriter',
                        options={'strings_to_urls': False})

data_test_1.to_excel(writer, sheet_name='Sheet1', index = False)
writer.save()